In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re
import sys 

import cptac
import cptac.utils as u

In [2]:
#load GBM data 
brain = cptac.Gbm()
gene = "EGFR"

In [3]:
Hnscc = cptac.Hnscc()

In [4]:
 #Step 1 - Create df
df2_Hnscc = Hnscc.join_omics_to_mutations(mutations_genes="EGFR", omics_df_name="proteomics")
 #Step 2 - Create binary column 
df2_Hnscc['binary_mutations'] = np.where(
        df2_Hnscc["EGFR"+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')
# Step 3 - Format the dataframe correctly for the T-test(just omics and binary columns for tumors)
tumors = df2_Hnscc[df2_Hnscc.Sample_Status == 'Tumor'] #drop Normal samples
columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
mut_status = tumors.drop(columns_to_drop, axis = 1)
mut_status_Hnscc = mut_status.dropna(axis=1,how='all')
mut_status_Hnscc

Name,A1BG_proteomics,A1CF_proteomics,A2M_proteomics,A2ML1_proteomics,AAAS_proteomics,AACS_proteomics,AADAC_proteomics,AAGAB_proteomics,AAK1_proteomics,AAMDC_proteomics,...,ZRANB2_proteomics,ZSCAN18_proteomics,ZSCAN22_proteomics,ZSWIM8_proteomics,ZW10_proteomics,ZWILCH_proteomics,ZYG11B_proteomics,ZYX_proteomics,ZZEF1_proteomics,binary_mutations
Sample_ID,,,,,,,,,,,,,,,,,,,,,
S001,209008.6094,NaN,3.169588e+05,127867.55470,61198.50781,54445.06641,NaN,138961.29690,143747.10940,34986.925780,...,62996.69922,NaN,NaN,NaN,50810.33984,NaN,4094.607666,480512.5625,37413.47656,Wildtype
S002,572630.5625,NaN,5.605086e+05,262365.81250,30010.43555,71837.50781,69098.47656,75495.77344,97025.68750,123058.085900,...,58077.59766,7711.908691,1.060632e+06,NaN,36131.23438,NaN,1639.182739,469727.1563,NaN,Wildtype
S003,595493.8125,NaN,1.107243e+06,25573.76563,19343.25391,73388.98438,NaN,68165.36719,134949.07810,87318.882810,...,53673.23438,3136.051270,1.264901e+05,NaN,48966.75391,NaN,3454.098145,610486.0000,28222.31836,Wildtype
S004,375582.6250,NaN,4.859903e+05,373164.75000,18545.10938,111085.10160,NaN,67968.65625,168306.04690,28307.119140,...,58258.71875,6938.365723,NaN,NaN,32046.61133,14693.20313,NaN,802913.1250,19900.98047,Wildtype
S005,292367.3125,NaN,3.238957e+05,115465.53130,34675.20703,153974.01560,NaN,51690.14844,94351.96875,63841.089840,...,53108.20703,2420.186279,2.646007e+05,NaN,32376.48438,30586.55859,2526.389893,598533.5000,12320.25977,Wildtype
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S106,360265.2500,NaN,5.847833e+05,22158.80859,40201.50781,76163.73438,NaN,128036.94530,106205.28910,57411.007810,...,76179.98438,4790.901367,8.336061e+05,6896.625000,33699.50781,19198.66016,NaN,769613.3125,12721.75879,Wildtype
S107,276656.6875,NaN,2.850874e+05,75872.71875,44822.48438,64772.04688,NaN,100212.83590,85217.65625,63943.175780,...,82209.05469,NaN,NaN,4862.116211,45518.02734,76605.22656,1037.856567,559177.6250,21917.46484,Wildtype
S108,248185.9688,NaN,4.693888e+05,43132.57813,45770.72656,166215.85940,NaN,45842.58594,137843.82810,44870.621090,...,77744.90625,NaN,NaN,8308.628906,22506.18164,36435.79688,1090.573486,337315.2500,21635.92188,Wildtype


In [5]:
#Step 1 - Create df
df1_GBM = brain.join_omics_to_mutations(mutations_genes="EGFR", omics_df_name="proteomics", omics_genes = "EGFR")
 #Step 2 - Create binary column 
df1_GBM['binary_mutations'] = np.where(
        df1_GBM["EGFR"+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')
# Step 3 - Format the dataframe correctly for the T-test(just omics and binary columns for tumors)
tumors = df1_GBM[df1_GBM.Sample_Status == 'Tumor'] #drop Normal samples
columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
mut_status = tumors.drop(columns_to_drop, axis = 1)
mut_status2 = mut_status.dropna(axis=1,how='all')
mut_status2

Name,EGFR_proteomics,binary_mutations
Sample_ID,,
S001,1.054962,Wildtype
S002,2.219562,Mutated
S003,0.137329,Wildtype
S004,-0.768478,Wildtype
S005,-1.249435,Wildtype
...,...,...
S100,0.356865,Mutated
S101,1.623039,Wildtype
S102,-0.381447,Wildtype


In [6]:

col_list = list(mut_status2.columns)
col_list.remove('binary_mutations')

print("Doing t-test comparisons for GBM\n")

# Call wrap_ttest, pass in formatted dataframe
wrap_results = u.wrap_ttest(mut_status2, 'binary_mutations', col_list, return_all=True)

Doing t-test comparisons for GBM

1 significant comparisons!


In [7]:
#function for context-dependent trans effects. Multi panel figure
def (df1, df1_name, df2, df2_name):
    
    

SyntaxError: invalid syntax (<ipython-input-7-1c37ba2f3dbe>, line 2)